In [1]:
import sys, os

import numpy as np
import pandas as pd
import shelve
from src.neuron import *
from src.utils import *
from src.constants import * 
from src.network import *
from src.validation import *
from src.viz import *
from src.genetic_algorithm import *
from copy import copy
from datetime import datetime

In [3]:
all_neurons = create_neurons()

# === Preparing Network === 
splits, input_waves, alpha_array = create_experiment()

# === Defining Criteria === 
criteria_dict = define_criteria()

jh_weights= create_dna_string(new_jh_weights, ACTIVE_SYNAPSES)
curr_dna = jh_weights
dna_matrix = load_dna(curr_dna) 

condition = 'experimental'

prepare_neurons(
            neurons=all_neurons,
            cue_wave=input_waves[0],
            go_wave=input_waves[1],
            control=True if condition == 'control' else False
        )
        
# run_network(
#             neurons=all_neurons,
#             weight_matrix=dna_matrix,
#             alpha_array=alpha_array,
#         )
neurons = all_neurons
weight_matrix = dna_matrix

#COPIED FROM NETWORK.PY FOR MODIFICATION
spikers = np.zeros(len(neurons))  # Initialize with zeros for all neurons

n = neurons[1]
print(n.name)

# Running the network
for t in range(1, TMAX):
    if 1028<t<1135:
        print('inspection time!')
    v0 = n.V
    u0 = n.u
    # Distributing alphas
    if np.any(spikers):
        alpha = alpha_fit(alpha_array, t, TMAX)

        for i, post in enumerate(neurons):
            post.input += alpha * np.dot(spikers, weight_matrix)[i]

    input = n.input[t]

    # Collecting spikes from all neurons from previous time step
    spikers = np.zeros(len(neurons))  # Initialize with zeros for all neurons

    for i, neu in enumerate(neurons):
        neu.hist_V[t], neu.hist_u[t], neu.spike_times[t - 1] = neu.update(I_ext=neu.input[t], sigma=0)
        spikers[i] = 1.0 if neu.spiked else 0.0  # Use 1.0 for spikes instead of True
        if neu.spiked:
            neu.hist_V[t - 1] = neu.vpeak

    v1 = n.V
    u1 = n.u
    if t<10:
        print(f'time | (V0,u0) | input | (V1,u1)')
        print(f'{t} | ({v0:.4f},{u0:.4f}) | {input} | ({v1:.4f},{u1:.4f})')
        if n.spiked:
            print(f'***************Spiked! (at end of time step {t})********************')


MSN1
time | (V0,u0) | input | (V1,u1)
1 | (-80.0000,0.0000) | 0.0 | (-78.6000,0.0000)
time | (V0,u0) | input | (V1,u1)
2 | (-78.6000,0.0000) | 0.0 | (-78.7008,-0.2800)
time | (V0,u0) | input | (V1,u1)
3 | (-78.7008,-0.2800) | 0.0 | (-78.6906,-0.5370)
time | (V0,u0) | input | (V1,u1)
4 | (-78.6906,-0.5370) | 0.0 | (-78.6859,-0.7936)
time | (V0,u0) | input | (V1,u1)
5 | (-78.6859,-0.7936) | 0.0 | (-78.6810,-1.0484)
time | (V0,u0) | input | (V1,u1)
6 | (-78.6810,-1.0484) | 0.0 | (-78.6761,-1.3018)
time | (V0,u0) | input | (V1,u1)
7 | (-78.6761,-1.3018) | 0.0 | (-78.6713,-1.5535)
time | (V0,u0) | input | (V1,u1)
8 | (-78.6713,-1.5535) | 0.0 | (-78.6665,-1.8037)
time | (V0,u0) | input | (V1,u1)
9 | (-78.6665,-1.8037) | 0.0 | (-78.6617,-2.0524)
inspection time!
inspection time!
inspection time!
inspection time!
inspection time!
inspection time!
inspection time!
inspection time!
inspection time!
inspection time!
inspection time!
inspection time!
inspection time!
inspection time!
inspection ti

In [11]:
print(*alpha_array)

0.0876 0.1695 0.246 0.3172 0.3835 0.4451 0.5023 0.5552 0.6041 0.6492 0.6908 0.7288 0.7637 0.7955 0.8244 0.8505 0.874 0.8951 0.9138 0.9304 0.9449 0.9574 0.9681 0.9771 0.9845 0.9903 0.9947 0.9977 0.9994 1.0 0.9995 0.9979 0.9953 0.9919 0.9876 0.9825 0.9767 0.9702 0.9631 0.9554 0.9472 0.9384 0.9293 0.9197 0.9098 0.8995 0.8889 0.8781 0.867 0.8557 0.8442 0.8325 0.8207 0.8088 0.7968 0.7847 0.7725 0.7603 0.748 0.7358 0.7235 0.7113 0.699 0.6868 0.6747 0.6626 0.6506 0.6387 0.6268 0.6151 0.6034 0.5918 0.5804 0.569 0.5578 0.5467 0.5358 0.5249 0.5142 0.5037 0.4932 0.483 0.4728 0.4628 0.453 0.4433 0.4337 0.4244 0.4151 0.406 0.3971 0.3883 0.3796 0.3711 0.3628 0.3546 0.3465 0.3386 0.3309 0.3232 0.3158 0.3084 0.3013 0.2942 0.2873 0.2805 0.2739 0.2674 0.261 0.2548 0.2487 0.2427 0.2368 0.2311 0.2255 0.22 0.2146 0.2093 0.2042 0.1991 0.1942 0.1894 0.1847 0.1801 0.1756 0.1712 0.1669 0.1627 0.1586 0.1546 0.1507 0.1468 0.1431 0.1395 0.1359 0.1324 0.129 0.1257 0.1224 0.1193 0.1162 0.1132 0.1102 0.1074 0.1046 0

In [9]:
0.088*220

19.36